In [49]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Load the datasets
result  = pd.read_csv('data/result.csv')
result['Gender'] = result['Gender'].map({'M': 1, 'F': 0})
# Ensure correct data types
result['score'] = result['score'].astype(float)

result.head()




,Applicant ID,School Name,GPA,Degree,Location,Gender,Veteran status,Work authorization,Disability,Ethnicity,...,Start 1,End 1,Role 2,Start 2,End 2,Role 3,Start 3,End 3,score,prediction
0,0,Providence University,3.81,Bachelors,Providence,1.0,0,0,0.0,0.0,...,5/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.35,1
1,1,Providence University,3.81,Bachelors,Providence,1.0,0,0,0.0,1.0,...,5/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.17,0
2,2,Providence University,3.81,Bachelors,Providence,1.0,0,0,0.0,NaN,...,5/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.37,1
3,3,Providence University,3.81,Bachelors,Providence,1.0,0,0,1.0,0.0,...,5/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.20,1
4,4,Providence University,3.81,Bachelors,Providence,1.0,0,0,1.0,1.0,...,5/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.56,0


In [34]:
sensitive_attr = ['Gender', 'Veteran status', 'Work authorization', 'Disability', 'Ethnicity']

In [45]:
def score_metrics(data, attr):
    """
    Calculate fairness metrics for the 'score' column based on a specified attribute.
    
    Parameters:
    - data (DataFrame): The DataFrame containing the data.
    - attr (str): The column name of the attribute based on which to calculate fairness (e.g., 'Gender').
    
    Assumes:
    - 'score' column exists in the DataFrame.
    - The attribute column contains binary values 0 and 1.
    """
    # Calculate Disparate Impact
    average_score_privileged = data[data[attr] == 1]['score'].mean()
    average_score_unprivileged = data[data[attr] == 0]['score'].mean()
    disparate_impact = average_score_unprivileged / average_score_privileged
    print(f"Disparate Impact based on {attr}:", disparate_impact)

    # Calculate Statistical Parity Difference
    statistical_parity_difference = average_score_unprivileged - average_score_privileged
    print(f"Statistical Parity Difference based on {attr}:", statistical_parity_difference)

    # Calculate Balanced Accuracy for Regression (using RMSE)
    overall_mean_score = data['score'].mean()
    # mse_privileged = mean_squared_error(data[data[attr] == 1]['score'], 
    #                                     np.full(data[data[attr] == 1].shape[0], overall_mean_score))
    # mse_unprivileged = mean_squared_error(data[data[attr] == 0]['score'], 
    #                                       np.full(data[data[attr] == 0].shape[0], overall_mean_score))
    # balanced_accuracy_regression = 1 - (np.sqrt(mse_privileged) + np.sqrt(mse_unprivileged)) / 2
    mae_privileged = mean_absolute_error(data[data[attr] == 1]['score'], 
                                     np.full(data[data[attr] == 1].shape[0], overall_mean_score))
    mae_unprivileged = mean_absolute_error(data[data[attr] == 0]['score'], 
                                        np.full(data[data[attr] == 0].shape[0], overall_mean_score))

    # Optionally calculate R-squared for each group
    r2_privileged = r2_score(data[data[attr] == 1]['score'], 
                            np.full(data[data[attr] == 1].shape[0], overall_mean_score))
    r2_unprivileged = r2_score(data[data[attr] == 0]['score'], 
                            np.full(data[data[attr] == 0].shape[0], overall_mean_score))
    print("Mean Absolute Error (Privileged):", mae_privileged)
    print("Mean Absolute Error (Unprivileged):", mae_unprivileged)
    print("R-squared (Privileged):", r2_privileged)
    print("R-squared (Unprivileged):", r2_unprivileged)

    # Calculate Average Odds Difference based on residuals
    residuals_privileged = data[data[attr] == 1]['score'] - overall_mean_score
    residuals_unprivileged = data[data[attr] == 0]['score'] - overall_mean_score
    average_odds_difference = abs(residuals_unprivileged.mean() - residuals_privileged.mean())
    print(f"Average Odds Difference (Regression) based on {attr}:", average_odds_difference)


In [28]:
score_metrics(result, "Ethnicity")

Disparate Impact based on Ethnicity: 1.0275380738956896
Statistical Parity Difference based on Ethnicity: 0.13722222222222147
Balanced Accuracy (Regression) based on Ethnicity: -1.9292518058782906
Average Odds Difference (Regression) based on Ethnicity: 0.13722222222222227


In [41]:
def eval_metrics(data, attr):
    """
    Calculate fairness metrics for the 'prediction' column based on a specified attribute.
    
    Parameters:
    - data (DataFrame): The DataFrame containing the data.
    - attr (str): The column name of the attribute based on which to calculate fairness (e.g., 'Gender').
    
    Assumes:
    - 'prediction' column exists in the DataFrame.
    - The attribute column contains binary values 0 and 1.
    """
    # Calculate Disparate Impact
    selection_rate_privileged = data[data[attr] == 1]['prediction'].mean()
    selection_rate_unprivileged = data[data[attr] == 0]['prediction'].mean()
    disparate_impact = selection_rate_unprivileged / selection_rate_privileged
    print(f"Disparate Impact based on {attr}:", disparate_impact)

    # Calculate Statistical Parity Difference
    statistical_parity_difference = selection_rate_unprivileged - selection_rate_privileged
    print(f"Statistical Parity Difference based on {attr}:", statistical_parity_difference)

    # Calculate Equal Opportunity Difference
    # Note: Correct calculation of TPRs should involve actual and predicted labels. Here we assume 'prediction' is the actual label for simplicity.
    cm_privileged = confusion_matrix(data[data[attr] == 1]['prediction'], data[data[attr] == 1]['prediction'])
    cm_unprivileged = confusion_matrix(data[data[attr] == 0]['prediction'], data[data[attr] == 0]['prediction'])

    tpr_privileged = cm_privileged[1, 1] / (cm_privileged[1, 0] + cm_privileged[1, 1])
    tpr_unprivileged = cm_unprivileged[1, 1] / (cm_unprivileged[1, 0] + cm_unprivileged[1, 1])
    equal_opportunity_difference = tpr_unprivileged - tpr_privileged
    print(f"Equal Opportunity Difference based on {attr}:", equal_opportunity_difference)




In [37]:
eval_metrics(result, "Ethnicity")

Disparate Impact based on Ethnicity: 1.090909090909091
Statistical Parity Difference based on Ethnicity: 0.01666666666666669
TPR Privileged based on Ethnicity: 1.0
Equal Opportunity Difference based on Ethnicity: 0.0


In [48]:
for attr in sensitive_attr:
    print(f"Metrics for {attr}")
    score_metrics(result, attr)
    print("\n")

Metrics for Gender
Disparate Impact based on Gender: 0.9848074875798438
Statistical Parity Difference based on Gender: -0.07611111111111146
Mean Absolute Error (Privileged): 2.572015020576132
Mean Absolute Error (Unprivileged): 2.490090946502058
R-squared (Privileged): -0.00013386002675597197
R-squared (Unprivileged): -0.00021342060485673997
Average Odds Difference (Regression) based on Gender: 0.0761111111111111


Metrics for Veteran status
Disparate Impact based on Veteran status: 1.0194429413179413
Statistical Parity Difference based on Veteran status: 0.09581481481481458
Mean Absolute Error (Privileged): 2.578026886145404
Mean Absolute Error (Unprivileged): 2.512111111111111
R-squared (Privileged): -0.0002594992913607097
R-squared (Unprivileged): -0.00027743276929159677
Average Odds Difference (Regression) based on Veteran status: 0.09581481481481482


Metrics for Work authorization
Disparate Impact based on Work authorization: 1.047854584254249
Statistical Parity Difference based 

In [47]:
for attr in sensitive_attr:
    print(f"Metrics for {attr}")
    eval_metrics(result, attr)
    print("\n")

Metrics for Gender
Disparate Impact based on Gender: 0.6507936507936508
Statistical Parity Difference based on Gender: -0.1222222222222222
Equal Opportunity Difference based on Gender: 0.0


Metrics for Veteran status
Disparate Impact based on Veteran status: 0.8571428571428572
Statistical Parity Difference based on Veteran status: -0.029629629629629617
Equal Opportunity Difference based on Veteran status: 0.0


Metrics for Work authorization
Disparate Impact based on Work authorization: 0.9622641509433961
Statistical Parity Difference based on Work authorization: -0.007407407407407418
Equal Opportunity Difference based on Work authorization: 0.0


Metrics for Disability
Disparate Impact based on Disability: 0.9411764705882354
Statistical Parity Difference based on Disability: -0.0111111111111111
Equal Opportunity Difference based on Disability: 0.0


Metrics for Ethnicity
Disparate Impact based on Ethnicity: 1.090909090909091
Statistical Parity Difference based on Ethnicity: 0.0166666

result.columns